In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.2 MB/s 
     |████████████████████████████████| 182 kB 56.6 MB/s 
     |████████████████████████████████| 7.6 MB 44.0 MB/s 


In [ ]:
import transformers

In [ ]:
from transformers import ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']

In [ ]:
tokenizer('안녕하세요')

{'input_ids': [2, 11655, 4279, 8553, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
tokenizer.tokenize("뷁뷉녊. 안녕 오늘은 날씨가 참 좋네요")

['[UNK]', '.', '안녕', '오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요']

토크나이저 훈련

1. 어떤 기법을 사용할지
2. vocab_size 결정 >> 작으면 메모리 이득, 크면 의미를 잘 표현할 수 있따
3. corpus >> 통계적으로 분석, vocab

tokenizer에 vocab 수동으로 직접 추가

vocab 300 ~ 350 [UNK1], [UNK2], [UNK3] ...


In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
tokenizer.convert_ids_to_tokens([''])

ValueError: ignored

In [ ]:
tokenizer.convert_tokens_to_ids(['[UNK]', '.', '안녕', '오늘', '##은', '날씨', '##가', '참', '좋', '##네', '##요'])

[100, 1012, 100, 100, 100, 100, 100, 100, 100, 100, 100]

In [ ]:
tokenizer.convert_ids_to_tokens([0,1,2,3, 155,156, 30000, 30001, 350001])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '※', '‼', '킬로그램', '태백산', '[UNK]']

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
tokenizer.tokenize('안녕하세요')

['ᄋ',
 '##ᅡ',
 '##ᆫ',
 '##ᄂ',
 '##ᅧ',
 '##ᆼ',
 '##ᄒ',
 '##ᅡ',
 '##ᄉ',
 '##ᅦ',
 '##ᄋ',
 '##ᅭ']

In [ ]:
tokenizer.vocab_size

30522

In [ ]:
multi_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
multi_tokenizer.tokenize('안녕하세요')

['안', '##녕', '##하', '##세', '##요']


multi : '안녕' << vocab에 포함이 안되어잇음
eletra : '안녕' 이 vocab에 포함되어있음

In [ ]:
multi_tokenizer.vocab_size

119547

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
tokenizer.convert_ids_to_tokens([111,22,33])

['Ġ-', 'Ġ"', 'Ġhave']

In [ ]:
tokenizer.convert_tokens_to_ids('hahah')

3

In [ ]:
'Webtest라고 불리는 [MASK] webpage로 외부적인 감독 없이 QA, 기계 번역, 문맥읽기, 요약 같은 걸 하도록 만들었다. 55F1, CoQA dataset을 QA했을 때 성능이 3/4 (SOTA)이다. (training exapmles없이)'

'Webtest라고 불리는 수백만의 webpage로 외부적인 감독 없이 QA, 기계 번역, 문맥읽기, 요약 같은 걸 하도록 만들었다. 55F1, CoQA dataset을 QA했을 때 성능이 3/4 (SOTA)이다. (training exapmles없이)'

In [ ]:
input = ' ~~~ [MASK] ~~~'

output = '원래문장'

라벨링 데이터를 >> Pretraining 



오늘 날씨가 너무 좋다. // 긍정

>> '오늘 날씨가 너무 좋다'는 [긍정/부정] 문장이다. 정답: 긍정




**3가지 종류 언어모델**

>> 언어에 대한 통계적 이해를 가진 모델

1. Decoder-only 모델 Auto regressive model (GPT1, GPT2, GPT3, ..
   - few shot learning 사용
   - 단점: 모델이 너무 큼
   - 점점 더 모델이 커지고있음(모델이 크고, 데이터를 많이 넣을수록 성능이 좋다.)

2. Encoder-only 모델 Auto encoder (BERT, RoBERTa, Electra, Albert, ..
   - input을 vector 로 바꿔준다.
   - vector는 의미와 관련이 있다. 

   - Pre-training(사전학습, 비지도학습) >> fine-tuning(지도학습)
   - Downstream task(번역, 문장 분류, 질의응답, ...)
   - Why? 
      1. 성능이 기존 방식보다 훨씬 좋다
      2. 소량의 데이터로도 학습이 훨씬 빨리된다



3. Encoder-Decoder 모델 >> Transformer 모델과 구조는 동일(BART, T5)

   - Pre-training(사전학습, 비지도학습) >> fine-tuning(지도학습)
   - 생성 관련 Task 사용


Github 코파일럿

AIHUB 감정분석 데이터셋
(감정 6가지로 분류)
(문장1, 분노)
(문장2, 슬픔)
,,
koelectra 모델 적용, 감정분석 

Encoder 모델은 생성 task 불가능, 추출은 가능
Decoder 모델 가능


챗봇을 만들고 싶다.
시나리오 1. Encoder 모델로 챗봇을 만드려면..?
>> (답변1, 답변2, 답변3 , ...)
질문 > 뽑아낸다

시나리오 2. Decoder 모델을 사용 > 좀 더 사람같은 느낌, 위험성, 엉뚱한 대답이 나올 때가 있음




nlp
훈련
배치 사이즈 클수록 좋음,




한국어 Pretrained model

- koelectra
- klue-roberta
- korbert


Representation from Transformer

embedding

고양이 -> 1 -> [1,1]  , 낙타 -> 2 -> [0.5, 1.5] , 의자 -> 3-> [-0.3, -3]

고양이-개
고양이-낙타 1
낙타-의자 2

Attention is All you Need

Attention 이 뭐지?


e


i Q(1,1,2) K(1,3,0) V(1,2,3)


am Q(3,0,2) K(1,1,0) V(1,0,1)

<!-- 
a Q(1,1,2) K(1,3,0) V(1,2,3)


student Q(1,1,2) K(1,3,0) V(1,2,3) -->

Q(1,1,0)


Decoder에서 추론 시점에 Encoder 토큰을 얼마나 참조하냐?

Self-attention


2017년 google Attention is All you Need , transformer
NLP가 급격히 성장하기 시작했다!

2018~2019년 BERT >> transformer에서 Encoder만 땐 모델

사전학습모델 


RNN


CELL1 > CELL2 > CELL3 > ..


문장 전체의 의미를 나타내는 벡터가 뭐지?
>> 각 토큰이 나타내는 벡터를 평균한다.



질의응답


[CLS] 포메라니안은 어떤걸 좋아해? [SEP] 포메라니안은 독일 태생의 사냥견으로, 노는걸 좋아한다. [SEP]


text2image 모델


BERT >> 문장을 벡터로 만들어준다

>> 벡터를 이미지로 만든다

